# Taking a Look at MAESTRO Data with Chromagen

The MAESTRO dataset includes WAV/MIDI file pairs of piano compositions, and metadata in the form of .csv and .json files. My goal is to use the Chromagen module for an exploratory k-means clustering analysis of a set of WAV files. I'll ignore the MIDI files in this demo, and take the output of Chromagen.chromaweights() as our feature vector for each file.

In [71]:
import numpy as np #for linear algebra
import pandas as pd #for nice databases
from scipy.io.wavfile import read as wavread #we can read wavfiles this way
import zipfile as z #to get to the dataset
import chromagen as cg

In [58]:
#since the dataset .zip has more than one file in it, we can't create a dataframe directly from the zip
file = z.ZipFile("D:/Datasets/maestro-v1.0.0.zip")
#extracted just the .csv holding our metadata
file.extract("maestro-v1.0.0/maestro-v1.0.0.csv", path = "D:/Datasets")
#now create a pandas dataframe using the csv
df = pd.read_csv("D:/Datasets/maestro-v1.0.0/maestro-v1.0.0.csv")

In [59]:
##let's take a look at the metadata
df.head()

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
2,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
3,Alexander Scriabin,"3 Etudes, Op. 65",test,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508
4,Alexander Scriabin,"5 Preludes, Op.15",train,2009,2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MI...,2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MI...,400.557826


It would take some time to extract all of these files, and we don't need a huge dataset for our basic exploration. Let's take a subset of the 954 WAV files listed as "training" files to narrow the field a bit. I'll choose only the WAV files whose corresponding MIDI file is between 100 and 200 seconds.

In [60]:
df.query('split == "train" and 100<duration<200', inplace = True)

In [61]:
df

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
8,Alexander Scriabin,"Etude Op. 8, No. 13",train,2009,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MI...,2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MI...,167.085837
9,Alexander Scriabin,"Etude in D-flat Major, Op. 8 No. 10",train,2011,2011/MIDI-Unprocessed_15_R1_2011_MID--AUDIO_R1...,2011/MIDI-Unprocessed_15_R1_2011_MID--AUDIO_R1...,102.007110
34,Claude Debussy,"""Les collines d'Anacapri"" from Preludes, Book I",train,2008,2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...,2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...,166.495560
62,Claude Debussy,"Ondine from Preludes, Book II",train,2008,2008/MIDI-Unprocessed_10_R3_2008_01-05_ORIG_MI...,2008/MIDI-Unprocessed_10_R3_2008_01-05_ORIG_MI...,193.301089
68,Claude Debussy,"Preludes, Book II, III - La puerta del vino",train,2013,2013/ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_02_...,2013/ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_02_...,179.321402
...,...,...,...,...,...,...,...
1148,Sergei Rachmaninoff / György Cziffra,Flight of the Bumblebee,train,2006,2006/MIDI-Unprocessed_12_R1_2006_01-08_ORIG_MI...,2006/MIDI-Unprocessed_12_R1_2006_01-08_ORIG_MI...,114.583219
1149,Sergei Rachmaninoff / Vyacheslav Gryaznov,Italian Polka,train,2009,2009/MIDI-Unprocessed_04_R1_2009_04-06_ORIG_MI...,2009/MIDI-Unprocessed_04_R1_2009_04-06_ORIG_MI...,182.654732
1156,Wolfgang Amadeus Mozart,"Sonata in B-flat Major, K. 281, First Movement",train,2011,2011/MIDI-Unprocessed_02_R1_2011_MID--AUDIO_R1...,2011/MIDI-Unprocessed_02_R1_2011_MID--AUDIO_R1...,183.087424
1178,Wolfgang Amadeus Mozart,"Sonata in F Major, K. 280, 1st mov.",train,2013,2013/ORIG-MIDI_03_7_6_13_Group__MID--AUDIO_09_...,2013/ORIG-MIDI_03_7_6_13_Group__MID--AUDIO_09_...,192.605310


In [62]:
for index, row in df.iterrows():
    filepath = row['audio_filename']
    file.extract("maestro-v1.0.0/" + filepath, path = "D:/Datasets")

In [72]:
X = pd.Series()
for index, row in df.iterrows():
    filepath = row['audio_filename']
    rate, data = wavread("D:/Datasets/maestro-v1.0.0/" + filepath)
    data = np.average(data, axis = 1)
    f, t, c = cg.stft(data,10000,rate, windowtype = "Hann")
    chrm = cg.chromagram(F_arr = f, Chi = c)
    x = cg.chromaweights(chrm)
    
    

C:\Users\jreif\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


AxisError: axis 1 is out of bounds for array of dimension 1

# Licensing

The MAESTRO dataset (v1.0.0) is made available by Google LLC under a [Creative Commons Attribution Non-Commercial Share-Alike 4.0 (CC BY-NC-SA 4.0) license](https://creativecommons.org/licenses/by-nc-sa/4.0/), and all new analysis of that unaltered data done in this notebook uses the same license. 

# Data Sources

The [MAESTRO v1.0.0 dataset](https://magenta.tensorflow.org/datasets/maestro#v100) used here was introduced in the following work:

Curtis Hawthorne, Andriy Stasyuk, Adam Roberts, Ian Simon, Cheng-Zhi Anna Huang,
  Sander Dieleman, Erich Elsen, Jesse Engel, and Douglas Eck. "Enabling
  Factorized Piano Music Modeling and Generation with the MAESTRO Dataset."
  In International Conference on Learning Representations, 2019.